# ARIMA model for Tesla Stock

Nick Sarris

### Sources:

- http://people.duke.edu/~rnau/Notes_on_nonseasonal_ARIMA_models--Robert_Nau.pdf
    - https://people.duke.edu/~rnau/arimrule.htm

- http://www.michaeljgrogan.com/arima-model-statsmodels-python/

- https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/

## Pre Processing Data

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

import matplotlib.pyplot as plt
plt.style.use('ggplot')
#ggplot makes fun looking graphs

%matplotlib inline

In [13]:
fig_size = plt.rcParams["figure.figsize"]
 
# Prints: [8.0, 6.0]
print "Current size:", fig_size
 
# Set figure width to 12 and height to 9
fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

Current size: [12.0, 9.0]


In [20]:
#Import Data

TSLA = pd.read_csv('Documents/TSLA_10Y(2).csv')

TSLA.head()

IOError: File Documents/TSLA_10Y(2).csv does not exist

In [ ]:
TSLA.describe().T

In [ ]:
# Converting index to datetime object

TSLA.index = pd.to_datetime(TSLA.index)

In [ ]:
Open = TSLA['Open']
High = TSLA['High']
Low = TSLA['Low']
Close = TSLA['Close']
Volume = TSLA['Volume']

First, we inspect the data to see if we can eye any trends with fast python commands and visualizations. 
    We start by looking at every column of our Dataframe:

In [ ]:
#Pandas rolling method used 

Open.rolling(30).mean().plot(label = '30 day rolling mean')
Open.rolling(365).mean().plot(label = 'Yearly rolling mean')
Open.rolling(30).std().plot(label = '30 day rolling std')
Open.rolling(365).std().plot(label = 'Yearly rolling std')
plt.plot(Open, alpha = .70, label = 'Open')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Open Price of Tesla Stock (10Y)')

In [ ]:
High.rolling(30).mean().plot(label = '30 day rolling mean')
High.rolling(365).mean().plot(label = 'Yearly rolling mean')
High.rolling(30).std().plot(label = '30 day rolling std')
High.rolling(365).std().plot(label = 'Yearly rolling std')
plt.plot(High, alpha = .70, label = 'Low')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('High Price of Tesla Stock')

In [ ]:
Low.rolling(30).mean().plot(label = '30 day rolling mean')
Low.rolling(365).mean().plot(label = 'Yearly rolling mean')
Low.rolling(30).std().plot(label = '30 day rolling std')
Low.rolling(365).std().plot(label = 'Yearly rolling std')
plt.plot(Low, alpha = .70, label = 'Low')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Low Price of Tesla Stock')

In [ ]:
Close.rolling(30).mean().plot(label = '30 day rolling mean')
Close.rolling(365).mean().plot(label = 'Yearly rolling mean')
Close.rolling(30).std().plot(label = '30 day rolling std')
Close.rolling(365).std().plot(label = 'Yearly rolling std')
plt.plot(Close, alpha = .70, label = 'Open')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Close Price of Tesla Stock')

In [ ]:
Volume.rolling(30).mean().plot(label = '30 day rolling mean')
Volume.rolling(365).mean().plot(label = 'Yearly rolling mean')
Volume.rolling(30).std().plot(label = '30 day rolling std')
Volume.rolling(365).std().plot(label = 'Yearly rolling std')
plt.plot(Volume, alpha = .70, label = 'Volume')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Volume Price of Tesla Stock')

## Determining Stationarity

From these graphs we can start to notice important information for the ARIMA parameter decision process: the data is not stationary. That is:

- The mean changes with respect to time
- The variance changes with respect to time

In Arima, we would like for the mean and variance to stay constant. In order to smooth out the variance, we take the natural log of each column and run the statistical tests from above again to see if it was successful. We omit the Volume graph because the behavior of it's variance is more volatile than the others, thus making it a poor candidate for ARIMA modelling. 

In [ ]:
lnOpen = np.log(Open)
lnHigh = np.log(High)
lnLow = np.log(Low)
lnClose = np.log(Close)

Here the plot is shown to visualize each natural logged column

In [ ]:
lnOpen.plot(label = 'ln(Open)', alpha = 0.5)
lnHigh.plot(label = 'ln(High)', alpha = 0.5)
lnLow.plot(label = 'ln(Low)', alpha = 0.5)
lnClose.plot(label = 'ln(Close)', alpha = 0.5)

plt.legend()

Comparing with the graph below, it looks the same except the spikes aren't as big as the ones below, indicating we did smooth the variance. Looking at the rolling variances and means for the exponentially smoothed graphs, as was done above, would give definitive proof of that.

In [ ]:
TSLA.Open.plot(alpha = .50, label = 'Open')
TSLA.High.plot(alpha = .50, label = 'High')
TSLA.Low.plot(alpha = .50, label = 'Low')
TSLA.Close.plot(alpha = .50, label = 'Close')

plt.legend()

In [ ]:

lnOpen.rolling(30).var().plot(label = 'ln(Open)', alpha = 0.4)
lnHigh.rolling(30).var().plot(label = 'ln(High)', alpha = 0.4)
lnLow.rolling(30).var().plot(label = 'ln(Low)', alpha = 0.4)
lnClose.rolling(30).var().plot(label = 'ln(Close)', alpha = 0.4)

plt.hlines(lnOpen.rolling(30).var().mean(),
          lnOpen.rolling(30).var().index[0],
          lnOpen.rolling(30).var().index[-1], label = 'Mean')


plt.title('Monthly rolling variance')
plt.legend()

In [ ]:
lnOpen.rolling(30).var().describe()

In [ ]:
lnOpen.rolling(30).var().index[0]

In [ ]:
lnOpen.rolling(30).mean().plot(label = 'ln(Open)', alpha = 0.4)
lnHigh.rolling(30).mean().plot(label = 'ln(High)', alpha = 0.4)
lnLow.rolling(30).mean().plot(label = 'ln(Low)', alpha = 0.4)
lnClose.rolling(30).mean().plot(label = 'ln(Close)', alpha = 0.4)

plt.hlines(lnOpen.rolling(30).mean().mean(),
          lnOpen.rolling(30).mean().index[0],
          lnOpen.rolling(30).mean().index[-1], label = 'Mean')


plt.title('Monthly rolling mean')
plt.legend()

In [ ]:
lnOpen.rolling(30).mean().describe()

Two important things should be noted:

- After smoothing the data, we notice that both graphs exhibit a sharp increase in the data between 2013 and 2014
- Relative to 10 years, all dataframe features in both graphs looks similar to one another
    - This implies that in our model, we can choose only one feauture column to make forecasts on future prices 
        of the stock. This is important, since we cannot use cross-sectional data in a univariate ARIMA process.
        

We would like to assume a linear relationship with the data, so we want to exlude the left portion of graph that contains the spike and the lower prices. Since there is still around four years of data after the exlusion, the sample size is still large enough to continue analyzing. 

Following from the second bullet point, from now on we will be only using the Open feature column. It is chosen because it is less volatile than the High and Low prices, making its variance more stationary than the Open. However, all three columns could have been used and the model would predict similar results.

In [ ]:
#Finding the date of the spike

lnOpen_sorted = lnOpen.rolling(30).var()

lnOpen_sorted.sort_values(ascending = False).head()

In [ ]:
#Finding date where spike ended

lnOpen_sorted['2013-05-31':'2013-06-30']

In [ ]:
plt.style.use('ggplot')

#Plotting remaining 30 day rolling variance from above graph, excluding aformetioned spike and data before it

lnOpen_var = lnOpen.rolling(30).var()['2014-06-25':'2018-12-03']

lnOpen_var.plot(label = '30 day rolling variance')
plt.hlines(lnOpen.rolling(30).var().mean(),
          lnOpen.rolling(30).var().index[890],
          lnOpen.rolling(30).var().index[-1], label = 'Unadjusted Mean', color = 'black')

plt.hlines(lnOpen['2014-06-25':'2018-12-03'].rolling(30).var().mean(),
          lnOpen.rolling(30).mean().index[890],
          lnOpen.rolling(30).mean().index[-1], label = 'Adjusted Mean', color = 'green')

plt.title('Open stock price, 30 day rolling variance, ADJ')
plt.legend()

In [ ]:
#Plotting remaining 30 day rolling mean from above graph, excluding aformetioned spike and data before it

lnOpen_mean = lnOpen.rolling(30).mean()['2014-06-25':'2018-12-03']

lnOpen_mean.plot(label = '30 day rolling mean')

plt.hlines(lnOpen.rolling(30).mean().mean(),
          lnOpen.rolling(30).mean().index[890],
          lnOpen.rolling(30).mean().index[-1], label = 'Unadjusted Mean', color = 'black')

plt.hlines(lnOpen['2014-06-25':'2018-12-03'].rolling(30).mean().mean(),
          lnOpen.rolling(30).mean().index[890],
          lnOpen.rolling(30).mean().index[-1], label = 'Adjusted Mean', color = 'green')

plt.title('Open stock price, 30 day rolling mean, ADJ')

plt.legend()

In [ ]:
#Plotting remaining lnOpen graph from above, excluding aformetioned spike and data before it

lnOpen_ = lnOpen['2014-06-25':'2018-12-03']

lnOpen_.plot()
plt.title('Open stock price ADJ')

Now that we have finished pre-processing and smoothing the data, we can make it stationary. 

Pandas has a method for this called diff(), which takes entry in Dataframe[x+1] and subtracts entry Dataframe[x] from it for every x in the length of the index of the Dataframe.

In [ ]:
lnOpen_.diff().describe()

In [ ]:
lnOpen_.diff().plot()

Note how the variance is approximately constant and the mean is approximately constant. 

**This indicates that the value d in ARIMA(p,d,q) is d = 1, since we only differenced our data once.**

To further test if our assumption that our data is stationary, we emply the Augmented Dicky Fuller Test.

In [ ]:
#adfuller is the Augmented Dicky Fuller test, which tests for stationarity

from statsmodels.tsa.stattools import adfuller

In [ ]:
lnOpen__ = lnOpen_.diff().dropna()

In [ ]:
#The important value is the second entry, which is called the p-value. The data is stationary if p < 0.05. Here, it is not.

adfuller(lnOpen_)

In [ ]:
#Here p has been rounded down which is why it has value 0.0, so p < 0.05 and thus the data is stationary after differencing.

adfuller(lnOpen__)

## Finding p,q for ARIMA(p,d,q) using ACF and PACF

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima_model import ARIMA

The ACF, or AutoCorrelation Function, tests the correlation between the first instance and the first lag, the first lag and the second lag, the second lag and the third lag, etc. 

The PACF, or Partial AutoCorrelation Function, tests the correlation between the first instance and the first lag, the first instance and the second lag, the first instance and the third lag, etc. 

Interpreting what each graph shows will indicate what p and q value we need for ARIMA(p,d,q) (the p in ARIMA(p,d,q) has nothing to do the with the p from the adfuller test above).

The p is associated with the AR model, or AutoRegressive model, which can be determined from the ACF. 

The q is associated with the MA mode, or the Moving Average model, which can be determined from the PACF.

Our hypothesis is that the stock price of today is most likely correlated with the stock price of yesterday, and not correlated with the stock price of last week, or last month. 

Thus, we are expecting that our data will follow the behavior of the ACF, and therefore be an AR model. 

In [ ]:
acf_1 =  acf(lnOpen_)[1:]

test_df = pd.DataFrame([acf_1]).T
test_df.columns = ['Pandas Autocorrelation']
test_df.index += 1
test_df.plot(kind = 'bar')

The linear relationship shown here supports the hypothesis. Each lag is significantly correlated with the one before it.

In [ ]:
pacf_1 =  pacf(lnOpen_)[1:]
#plt.plot(pacf_1)
#plt.show()

test_df = pd.DataFrame([pacf_1]).T
test_df.columns = ['Pandas Partial Autocorrelation']
test_df.index += 1
test_df.plot(kind='bar')

The results from the PACF support the hypothesis. There is high correlation between the first instance and the first lag, but little to no correlation exists between the first instance the the kth lag for k > 1. 

**From both graphs, we can conclude that our hypothesis was correct and the data has AutoRegressive behavior. Thus, our model is said to be an AR(1) model.**

In [ ]:
price_matrix=lnOpen__.as_matrix()
model = ARIMA(price_matrix, order=(1,1,0))
model_fit = model.fit(disp=0, trend = 'c')
print(model_fit.summary())

Above shows a summary of the results. Though I do not have the mathematical background to completely describe what everything in the summary means, the Real value is the coefficient of the AutoRegressive(1) equation. 

In [ ]:
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
residuals.plot(kind='kde')
print(residuals.describe())

Above shows the residual data remaining after extracting the patterns in the ACF and the PACF. The method used to get the residual data is predictably the model.fit().resid. 

Since data represents white noise is and is approximately distributed, we know that all patterns were extracted no other patterns can be found in the data. If trends still existed in the residuals, one would have to go back and change their p or q value until the shape in both graphs above are achieved. 


## Training and Testing Model

In [ ]:
from sklearn.metrics import mean_squared_error

The line of code below takes 2/3 of the data for training and the remaining 1/3 for testing. We iterate through every date and make a prediction, then compare it to the actual data. 

Since we took the natural log earlier to smooth the variance of the data out, we will now undo it with the np.exp() method. 

In [ ]:
X = np.exp(lnOpen_).values
size = int(len(X) * 2/3)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

for t in range(len(test)):
    model = ARIMA(history, order=(1,1,0))
    model_fit = model.fit(trend = 'c')    
    #predictions starts at index 1112 and ends at index 1119 (index 1119 is last entry), so
    #a weeks worth of prices of the stock
    output = model_fit.predict(1112, 1119, typ = 'levels')
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)

In [ ]:
from scipy import stats

In order to complete the analysis, confidence intervals will be plotted with the predictions graph in order to make better estimates for the forecast. The for-loop below finds each confidence interval for the last seven days in the dataframe, which are the indexes [362-373] (index 373 being the last entry).

In [ ]:
predictions__ = pd.DataFrame(predictions)

for i in range(7):
    
    n, min_max, mean, var, skew, kurt = stats.describe(predictions__[362+i:])
    std= np.sqrt(var)
    
    Q = []
    R = []
    S = []
    
    # Confidence intervals of 25%,66%, and 95%
    A = stats.norm.interval(0.95,loc=mean,scale=std)
    B = stats.norm.interval(0.66,loc=mean,scale=std)
    C = stats.norm.interval(0.25,loc=mean,scale=std)
    
    Q += C
    R += B
    S += A
    
    print(Q)
    print(R)
    print(S)
    print('\n')
    #prints out [(min), (max)]

Below is the graph of the predictions with the confidence intervals. Of the 1119 entries in the original data, 1/3 of the data was for training, which is 373. The x-axis below accounts for all 373 testing points below (excluding the first 300, which was sliced out of the code). The y-axis represents the price. The MSE is approximately 101, which is a good outcome for a first time ARIMA. Though the index is not in Datetime, the index should be the last 7 days of the data (2018/11/26 - 2018/12/03)

The floats in the plt.hlines are from the first set of Q,R,S lists from the for-loop above, since that index is when the ARIMA predictions started

In [ ]:
plt.plot(predictions[357:], color = 'black', label = 'rolling forecast prediction')
plt.plot(test[357:], color = 'purple', label = 'actual result')

plt.hlines(345.02695391, 9, 17, label = '25% Upper Conf Inv', color = 'red', alpha = 0.7)
plt.hlines(347.55748155, 9, 17, label = '66% Upper Conf Inv', color = 'orange', alpha = 0.7)
plt.hlines(351.56235615, 9, 17, label = '95% Upper Conf Inv', color = 'green', alpha = 0.7)

plt.hlines(342.48944684, 9, 17, label = '25% Lower Conf Inv', color = 'red', alpha = 0.7)
plt.hlines(339.9589192, 9, 17, label = '66% Lower Conf Inv', color = 'orange', alpha = 0.7)
plt.hlines(335.9540446, 9, 17, label = '95% Lower Conf Inv', color = 'green', alpha = 0.7)

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

## Conclusion

An interesting observation is that the confidence intervals are not centered around the black prediction line. This could suggest that over the course of the week, the price is expected to grow more than it is going to decrease. Since the actual value towards the end increased, it could support this conclusion.

Also, the spikes that dip below and above the confidence intervals show inaccuracies in the model. This suggests that the way stock prices behave cannot be captured solely through the ARIMA model. Perhaps there are other models and python libraries that can offer a better explanation for the erratic behavior such as *tweepy*, that conducts a sentiment analysis based off tweets. 

A lot of assumptions were made:

- The data was stationary
- No outside data influenced increases and decreases in stock price
- A linear relationship between time and the price of the stock was assumed

Though the model appears to be accurate, too many assumptions were made, and thus the bias is high. 

In the future, for more accuracy one should make auxiliary models and combine them with the outcome of this model for higher accuracy